

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/JohnSnowLabs/spark-nlp-workshop/edit/master/tutorials/streamlit_notebooks/healthcare/NER_PROFESSIONS_ES.ipynb)




# **Detect Professions and Occupations in Spanish text**

## 1. Colab Setup

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

Install dependencies

In [3]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

## 2. Start the Spark session

In [4]:
import json
import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

import sparknlp_jsl
from sparknlp_jsl.annotator import *

spark = sparknlp_jsl.start(license_keys['SECRET'])

spark

## 3. Select the DL model

In [8]:
# If you change the model, re-run all the cells below.
# Applicable models: meddroprof_scielowiki
MODEL_NAME = "meddroprof_scielowiki"

## 4. Some sample examples

In [9]:
# Enter examples to be transformed as strings in this list
text_list = [
    """Paciente varón de 42 años que acude al servicio de Urgencias de su hospital acompañado de las Fuerzas del Orden Público por presentar 
    una actitud hostil y desconfiada hacia su padre, permaneciendo aislado en su habitación desde hace un mes. 
    Se decide ingreso hospitalario en Salud Mental. 
    ANTECEDENTES Antecedentes personales y familiares: niega antecedentes personales y familiares de interés. 
    No reacciones alérgicas medicamentosas. 
    Nunca ha estado en tratamiento psiquiátrico, aunque, al parecer, según los datos aportados por familiares, viene presentando síntomas psicóticos desde hace al menos cinco años. 
    Se trata de un varón divorciado, con una hija de 13 años con la que refiere contacto esporádico. 
    Actualmente en desempleo . 
    Sus familiares verbalizan que desde hace unos cinco años –coincidiendo con la ruptura matrimonial, la pérdida de empleo Fuerzas del Orden Público."""
]

## 5. Define Spark NLP pipeline

In [10]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The model was trained with the bert_portuguese_base_cased embeddings,
# we need to it.
embeddings = WordEmbeddingsModel.pretrained("embeddings_scielowiki_300d", "es", "clinical/models")\
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

ner_model = MedicalNerModel.pretrained(MODEL_NAME, "es", "clinical/models") \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

embeddings_scielowiki_300d download started this may take some time.
Approximate size to download 351.2 MB
[OK!]
meddroprof_scielowiki download started this may take some time.
Approximate size to download 21.8 MB
[OK!]


## 6. Run the pipeline

In [11]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)

## 7. Visualize results

In [12]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)